# Collaborative Filtering(추천시스템:사용자 리뷰 기반 필터링)

## 한 영화에 나와 비슷한 평점을 준 유저들의 선호하는 다른 영화를 추천하는 시스템

In [3]:
import surprise
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

## 캐글에 주어진 평점 데이터 전처리하기

In [4]:
rating=pd.read_csv('ratings_small.csv')
rating.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
#min,max이용하여 rating scale 유추하기
rating['rating'].min()
rating['rating'].max()

5.0

## surprise에서 제공하는 reader 함수로 툴에 필요한 모양으로 전처리

In [6]:
reader=Reader(rating_scale=(0.5,5))

In [7]:
data=Dataset.load_from_df(rating[['userId','movieId','rating']],reader=reader)
data

In [8]:
svd=SVD(random_state=0)

## Validation기능을 통한 지표 평가하기(RMSE(root mean square error)와 MAE(mean absolute error)참고)

In [9]:
cross_validate(svd,data,measures=['RMSE','MAE'],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8912  0.9037  0.8942  0.8968  0.8989  0.8969  0.0043  
MAE (testset)     0.6867  0.6968  0.6911  0.6883  0.6914  0.6909  0.0035  
Fit time          5.30    5.65    5.24    5.40    5.47    5.41    0.14    
Test time         0.17    0.13    0.11    0.12    0.13    0.13    0.02    


{'test_rmse': array([0.8911656 , 0.9037118 , 0.89415267, 0.89677925, 0.89886005]),
 'test_mae': array([0.68666589, 0.69682768, 0.69110233, 0.68831005, 0.69139474]),
 'fit_time': (5.2969748973846436,
  5.654624938964844,
  5.242297172546387,
  5.3999950885772705,
  5.4706947803497314),
 'test_time': (0.16991806030273438,
  0.12526535987854004,
  0.11437010765075684,
  0.12247681617736816,
  0.12633395195007324)}

## surprise 패키지에서 제공하는 기능을 이용하여 trainset데이터로 svd 방식으로 리뷰 예측하기

In [10]:
trainset=data.build_full_trainset()
svd.fit(trainset)

밑에 있는 테이블을 보면 각 유저들이 영화에대해 어떻게 어떤 평점을 주었는지 볼 수 있는데 이러한 데이터를 기반으로 각 유저들이 해당 영화에 어떤 평점을 줄지 예측해보려고 합니다

In [17]:
rating[rating['userId']==1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


### userid=1 인 사람이 movie id=1029인 영화에 대해서 실제 평가는 3일때 어떻게 보여줄지 확인하기

In [26]:
svd.predict(1,1029)

Prediction(uid=1, iid=1029, r_ui=None, est=2.8814455446761933, details={'was_impossible': False})